# Proyek Analisis Data: Air-quality-dataset
- **Nama:** Arya Setia Pratama
- **Email:** aryasetia30@gmail.com
- **ID Dicoding:** aryasetia30

## Menentukan Pertanyaan Bisnis

- Bagaimana tren kualitas udara (PM2.5 dan PM10) di berbagai lokasi dari tahun 2013 hingga 2017?
- Faktor meteorologi apa yang paling mempengaruhi konsentrasi polutan di berbagai kota?
- Apakah ada perbedaan signifikan dalam kualitas udara antara lokasi yang berbeda pada waktu-waktu tertentu?

## Import Semua Packages/Library yang Digunakan

In [2]:
import pandas as pd
import glob

## Data Wrangling

### Gathering Data

In [1]:
import pandas as pd
import glob
import os

# Path ke folder tempat file CSV disimpan
path = 'data/'  # Sesuaikan dengan path folder kamu

# Mengambil semua file CSV yang ada di folder
all_files = glob.glob(os.path.join(path, "*.csv"))

# List untuk menyimpan DataFrame
dataframes = []

# Loop untuk memproses setiap file CSV
for file in all_files:
    try:
        # Membaca file CSV
        df = pd.read_csv(file)
        
        # Mendapatkan nama kota dari nama file dengan cara yang lebih fleksibel
        # Misalnya, 'PRSA_Data_Gucheng_20130301-20170228.csv' -> 'Gucheng'
        city_name = os.path.basename(file).split('_')[2]
        
        # Menambahkan kolom 'station' dengan nama kota
        df['station'] = city_name
        
        # Menambahkan dataframe ke dalam list
        dataframes.append(df)
    
    except Exception as e:
        print(f"Error membaca file {file}: {e}")
        continue

# Menggabungkan semua DataFrame menjadi satu
merged_df = pd.concat(dataframes, ignore_index=True)

# Memastikan direktori output ada
output_dir = 'dashboard'
os.makedirs(output_dir, exist_ok=True)

# Menyimpan DataFrame yang sudah digabungkan ke file CSV
output_path = os.path.join(output_dir, 'main_data.csv')
merged_df.to_csv(output_path, index=False)

print(f"File CSV berhasil digabungkan dan disimpan ke '{output_path}'")


File CSV berhasil digabungkan dan disimpan ke 'dashboard\main_data.csv'


**Insight:**
- Selama proses penggabungan data dari berbagai file CSV, ditemukan bahwa semua file memiliki struktur kolom yang seragam, yang memudahkan untuk melakukan penggabungan. Setiap file mencakup data dari tahun 2013 hingga 2017 dan berisi kolom yang sama seperti PM2.5, PM10, SO2, NO2, CO, O3, serta data meteorologi seperti suhu dan kelembapan. Konsistensi ini mempercepat proses data wrangling karena tidak perlu penanganan khusus untuk variasi kolom.
- Menambahkan kolom station ke dataset untuk menunjukkan lokasi data adalah langkah yang penting. Proses ini memberikan fleksibilitas dalam analisis, karena mempermudah pemisahan dan identifikasi data berdasarkan kota atau lokasi. Dengan adanya kolom ini, tren polusi dapat dengan mudah dianalisis secara geografis, serta memberikan kemampuan untuk membuat visualisasi perbandingan kualitas udara antar kota.

### Assessing Data

In [2]:
# Membaca data yang sudah digabungkan
merged_df = pd.read_csv('dashboard/main_data.csv')

# 1. Missing Value
print("=== Missing Values ===")
missing_values = merged_df.isnull().sum()
print(missing_values[missing_values > 0])  # Menampilkan kolom yang memiliki missing values

# 2. Invalid Value
print("\n=== Invalid Values ===")
# Misalnya, kita anggap nilai negatif pada kolom polutan dan temperatur tidak valid
invalid_values = {
    'PM2.5': merged_df[merged_df['PM2.5'] < 0].shape[0],
    'PM10': merged_df[merged_df['PM10'] < 0].shape[0],
    'SO2': merged_df[merged_df['SO2'] < 0].shape[0],
    'NO2': merged_df[merged_df['NO2'] < 0].shape[0],
    'CO': merged_df[merged_df['CO'] < 0].shape[0],
    'O3': merged_df[merged_df['O3'] < 0].shape[0],
    'TEMP': merged_df[merged_df['TEMP'] < -50].shape[0],  # Misalnya, suhu tidak boleh lebih kecil dari -50 derajat
}
print(pd.Series(invalid_values))

# 3. Duplicate Data
print("\n=== Duplicate Data ===")
duplicate_data = merged_df.duplicated().sum()
print(f"Jumlah baris duplikat: {duplicate_data}")

# 4. Outlier
print("\n=== Outliers ===")
# Menggunakan metode IQR untuk mendeteksi outliers
Q1 = merged_df[['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP']].quantile(0.25)
Q3 = merged_df[['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP']].quantile(0.75)
IQR = Q3 - Q1

# Menghitung batas bawah dan atas untuk mendeteksi outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Mendeteksi outliers
outliers = (merged_df[['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP']] < lower_bound) | (merged_df[['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP']] > upper_bound)
outlier_counts = outliers.sum()

print(outlier_counts[outlier_counts > 0])  # Menampilkan jumlah outliers per kolom


=== Missing Values ===
PM2.5     8739
PM10      6449
SO2       9021
NO2      12116
CO       20701
O3       13277
TEMP       398
PRES       393
DEWP       403
RAIN       390
wd        1822
WSPM       318
dtype: int64

=== Invalid Values ===
PM2.5    0
PM10     0
SO2      0
NO2      0
CO       0
O3       0
TEMP     0
dtype: int64

=== Duplicate Data ===
Jumlah baris duplikat: 0

=== Outliers ===
PM2.5    19142
PM10     14658
SO2      35566
NO2       7021
CO       28054
O3       16599
dtype: int64


**Insight:**
- Hasil analisis menunjukkan bahwa sejumlah besar kolom dalam dataset memiliki nilai yang hilang, dengan jumlah missing values tertinggi pada kolom CO (20,701), diikuti oleh O3 (13,277) dan NO2 (12,116). Ini menunjukkan bahwa data untuk polutan tertentu tidak lengkap, yang dapat memengaruhi akurasi analisis selanjutnya.
- Analisis outliers mengungkapkan bahwa terdapat jumlah yang signifikan dari outliers di beberapa kolom polutan, dengan jumlah tertinggi pada kolom SO2 (35,566) dan CO (28,054). Keberadaan outliers ini dapat mengindikasikan kejadian polusi ekstrem atau kesalahan pengukuran yang perlu diteliti lebih lanjut.
- Dari hasil assessing tersebut ada yang perlu ditangani dalam proses cleaning data selanjutnya, yaitu mengatasi missing value dan outliers.

### Cleaning Data

In [5]:
# Membaca data yang sudah digabungkan
merged_df = pd.read_csv('dashboard/main_data.csv')

# 1. Menangani Missing Values
# Mengganti missing values dengan median untuk kolom numerik
numeric_columns = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']

for column in numeric_columns:
    median_value = merged_df[column].median()
    merged_df[column] = merged_df[column].fillna(median_value)  # Tanpa inplace=True

# Menangani missing values pada kolom non-numerik
non_numeric_columns = ['wd']  # Tambahkan kolom non-numerik lainnya sesuai kebutuhan

for column in non_numeric_columns:
    mode_value = merged_df[column].mode()[0]  # Mendapatkan modus
    merged_df[column] = merged_df[column].fillna(mode_value)  # Mengganti missing values dengan modus

# 2. Menangani Outliers
# Menggunakan metode IQR untuk mendeteksi outliers
Q1 = merged_df[numeric_columns].quantile(0.25)
Q3 = merged_df[numeric_columns].quantile(0.75)
IQR = Q3 - Q1

# Menghitung batas bawah dan atas untuk mendeteksi outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Capping outliers
for column in numeric_columns:
    merged_df[column] = merged_df[column].clip(lower=lower_bound[column], upper=upper_bound[column])

# 3. Validasi Data
# Misalnya, memastikan nilai temperatur tidak lebih rendah dari -50
merged_df['TEMP'] = merged_df['TEMP'].clip(lower=-50)

# 4. Menghapus Duplikat
merged_df.drop_duplicates(inplace=True)

# 5. Menyimpan Data yang Sudah Dibersihkan
merged_df.to_csv('dashboard/cleaned_data.csv', index=False)

print("Data berhasil dibersihkan dan disimpan ke 'dashboard/cleaned_data.csv'")


Data berhasil dibersihkan dan disimpan ke 'dashboard/cleaned_data.csv'


**Insight:**
- Proses pengisian nilai yang hilang (missing values) dengan menggunakan median untuk kolom numerik dan modus untuk kolom non-numerik (seperti arah angin) sangat membantu dalam menjaga integritas data. Dengan mengganti missing values, dapat memastikan bahwa analisis lebih akurat dan representatif terhadap kondisi nyata, sehingga hasil analisis tidak terdistorsi oleh nilai yang hilang.
- Proses deteksi dan pengendalian outliers menggunakan metode IQR memberikan gambaran yang lebih bersih dan relevan tentang distribusi data kualitas udara. Capping outliers membantu mengurangi efek nilai ekstrem yang bisa mengganggu analisis dan interpretasi data kedepannya.

## Exploratory Data Analysis (EDA)

### Explore ...

**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1:

### Pertanyaan 2:

**Insight:**
- xxx
- xxx

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2